In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import pandas as pd
import os

from hyphen import Hyphenator
hyphe = Hyphenator('en_GB')
# hyphe.syllables('Destruction')


In [6]:
listFossilsGhost_Capital = ['FOSSIL_KABUTOPS', 'FOSSIL_AERODACTYL', 'MON_GHOST',]


In [7]:
def grabEvosMovesMon(pkmn, inputListOfLines): # edited
    index_start = inputListOfLines.index(f'{pkmn}EvosMoves:\n')
    index_temp = index_start
    subBlockToPrint = []
    while not ((inputListOfLines[index_temp]=='\tdb 0\n') and (inputListOfLines[index_temp+1]=='\n')):
        subBlockToPrint.append(inputListOfLines[index_temp])
        index_temp+=1
        if (index_temp)==len(inputListOfLines)-1:
            break
    subBlockToPrint.append('\tdb 0\n')
    subBlockToPrint.append('\n')
    return subBlockToPrint

In [8]:
def grabDexEntries(Pkmn, inputListOfLines): # edited
    index_start = inputListOfLines.index(f'{Pkmn}DexEntry:\n')
    species = inputListOfLines[index_start+1][len('\tdb "'):-len('@"\n')]
    height_dm = inputListOfLines[index_start+2][len('\tdb '):-len('\n')]
    weight_hg = inputListOfLines[index_start+3][len('\tdb '):-len('\n')]
    return species, height_dm, weight_hg

In [9]:
def lineifyDexEntries_OneLine(DexEntryWordByWord=['Hello', 'world'], lengthMax=18):
    stringTemp = ''
    lengthTemp = len(stringTemp)
    idxTemp = 0

    while True:
        if stringTemp=='':
            stringTempNext = stringTemp + DexEntryWordByWord[idxTemp]
        else:
            stringTempNext = stringTemp + ' ' + DexEntryWordByWord[idxTemp]

        # more than 18 chars
        if len(stringTempNext)>lengthMax:
            # check if just adding one space and two symbols is enough to go beyond limits:
            # previous string must have ended on a word, so a space is needed,
            # also if the word was one-letter long and fit, it would pass the test above,
            # so the best-case scenario is that the new word can be split from the first letter
            if len(stringTemp)+3>lengthMax:
                break
            else:
                # check if the next word can be split into syllables
                syllablesTemp = hyphe.syllables(DexEntryWordByWord[idxTemp])
                if len(syllablesTemp)>1:
                    # try add one syllable + space + hyphene at the time
                    for syllableIdx in range(len(syllablesTemp)):
                        syllable = syllablesTemp[syllableIdx]
                        # first syllable is different
                        if syllableIdx==0:
                            stringTempNext = stringTemp + ' ' + syllable + '-'
                        else:
                            stringTempNext = stringTemp[:-1] + syllable + '-'
                        # check if we went beyond the max allowed length
                        if len(stringTempNext)>lengthMax:
                            break
                        else:
                            # we succeded in adding a syllable to the line
                            stringTemp = stringTempNext
                            syllableIdx+=1
                    #break
                else:
                    # it can't be broken into syllables, henceforth (BEEEH) no hope, end of this line
                    break
        else:
            # all good, let's update the cycle
            stringTemp = stringTempNext
            idxTemp+=1
            # let's check if we ran out of words
            if idxTemp>=len(DexEntryWordByWord):
                wordsLeft = [] # mega ugly but works, a la GF
                break

        try:
            syllableIdx
        except:
            wordsLeft = DexEntryWordByWord[idxTemp:]
        else:
            if syllableIdx==0:
                wordsLeft = DexEntryWordByWord[idxTemp:]
            else:
                wordsLeft = syllablesTemp[syllableIdx:] + DexEntryWordByWord[idxTemp+1:]

    return stringTemp, wordsLeft

def lineifyDexEntries(DexEntry='Hello world', lengthMax=18, numberOfLinesMax=6):
    if len(DexEntry)>lengthMax*numberOfLinesMax:
        print('You will not be able to fit your Pokedex entry!')
        print(f'Your Dex entry is {len(DexEntry)} characters long, but with the current settings')
        print(f'you can have max {numberOfLinesMax} lines, each {lengthMax} characters long, for a total of {lengthMax*numberOfLinesMax} characters.')
        print('The constraints above can be changed my inputting different "numberOfLinesMax" and "lengthMax" parameters.')
    else:
        DexEntryWordByWord = DexEntry.rsplit()
        DexEntry_Lineified = []
        while len(DexEntryWordByWord)>0:
            lineHere, DexEntryWordByWord = lineifyDexEntries_OneLine(DexEntryWordByWord)
            DexEntry_Lineified.append(lineHere)
        if len(DexEntry_Lineified)>numberOfLinesMax:
            print('Sorry, I could not split the Dex entry into the required number of lines.')
            print('It may be possible to do so by changing (even against grammar rules) the hyphenation.')
            print('Here is the best I could do:')
        return DexEntry_Lineified
        
# TODO: take into account that "#MON" counts as 7 characters because it prints as "POKéMON"

In [10]:
def grabDexText(Pkmn, inputListOfLines): # edited
    index_start = inputListOfLines.index(f'_{Pkmn}DexEntry::\n')
    addendums = [1, 2, 3, 5, 6, 7]
    indexes = [index_start+addhere for addhere in addendums]
    dexEntry1Line = ''
    for idx in indexes:
        ender='' if (inputListOfLines[idx][len('\ttext "'):-len('"\n')])[-1:]=='-' else ' '
        dexEntry1Line += inputListOfLines[idx][len('\ttext "'):-len('"\n')]+ender
    return dexEntry1Line[:-1]

In [11]:
def checkAllSameLength(thingstocheck=[]):
    referenceLength = len(thingstocheck[0])
    for thinghere in thingstocheck:
        if len(thinghere)!=referenceLength:
            print('referenceLength =', referenceLength)
            print('len(thinghere) =', len(thinghere))
            return False
    return True

In [12]:
if False:
    listAllOriginalMons = ['Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle', 'Blastoise', 'Caterpie', 'Metapod', 'Butterfree', 'Weedle', 'Kakuna', 'Beedrill', 'Pidgey', 'Pidgeotto', 'Pidgeot', 'Rattata', 'Raticate', 'Spearow', 'Fearow', 'Ekans', 'Arbok', 'Pikachu', 'Raichu', 'Sandshrew', 'Sandslash', 'NidoranF', 'Nidorina', 'Nidoqueen', 'NidoranM', 'Nidorino', 'Nidoking', 'Clefairy', 'Clefable', 'Vulpix', 'Ninetales', 'Jigglypuff', 'Wigglytuff', 'Zubat', 'Golbat', 'Oddish', 'Gloom', 'Vileplume', 'Paras', 'Parasect', 'Venonat', 'Venomoth', 'Diglett', 'Dugtrio', 'Meowth', 'Persian', 'Psyduck', 'Golduck', 'Mankey', 'Primeape', 'Growlithe', 'Arcanine', 'Poliwag', 'Poliwhirl', 'Poliwrath', 'Abra', 'Kadabra', 'Alakazam', 'Machop', 'Machoke', 'Machamp', 'Bellsprout', 'Weepinbell', 'Victreebel', 'Tentacool', 'Tentacruel', 'Geodude', 'Graveler', 'Golem', 'Ponyta', 'Rapidash', 'Slowpoke', 'Slowbro', 'Magnemite', 'Magneton', 'Farfetchd', 'Doduo', 'Dodrio', 'Seel', 'Dewgong', 'Grimer', 'Muk', 'Shellder', 'Cloyster', 'Gastly', 'Haunter', 'Gengar', 'Onix', 'Drowzee', 'Hypno', 'Krabby', 'Kingler', 'Voltorb', 'Electrode', 'Exeggcute', 'Exeggutor', 'Cubone', 'Marowak', 'Hitmonlee', 'Hitmonchan', 'Lickitung', 'Koffing', 'Weezing', 'Rhyhorn', 'Rhydon', 'Chansey', 'Tangela', 'Kangaskhan', 'Horsea', 'Seadra', 'Goldeen', 'Seaking', 'Staryu', 'Starmie', 'MrMime', 'Scyther', 'Jynx', 'Electabuzz', 'Magmar', 'Pinsir', 'Tauros', 'Magikarp', 'Gyarados', 'Lapras', 'Ditto', 'Eevee', 'Vaporeon', 'Jolteon', 'Flareon', 'Porygon', 'Omanyte', 'Omastar', 'Kabuto', 'Kabutops', 'Aerodactyl', 'Snorlax', 'Articuno', 'Zapdos', 'Moltres', 'Dratini', 'Dragonair', 'Dragonite', 'Mewtwo', 'Mew', ]

    listFossilsGhost = ['FossilKabutops', 'FossilAerodactyl', 'MonGhost',]
    listFossilsGhost_Names = ['MissingNo.', 'MissingNo.', 'MissingNo.']
    listFossilsGhost_Names_Capital = ['MISSINGNO.', 'MISSINGNO.', 'MISSINGNO.']

    listAllOriginalMons_Capital = [x.upper() for x in listAllOriginalMons]
    for i in range(len(listAllOriginalMons_Capital)):
        if listAllOriginalMons_Capital[i] == "FARFETCH'D":
            listAllOriginalMons_Capital[i] = 'FARFETCHD'
        if listAllOriginalMons_Capital[i] == "NIDORANM":
            listAllOriginalMons_Capital[i] = 'NIDORAN_M'
        if listAllOriginalMons_Capital[i] == "NIDORANF":
            listAllOriginalMons_Capital[i] = 'NIDORAN_F'
        if listAllOriginalMons_Capital[i] == "MRMIME":
            listAllOriginalMons_Capital[i] = 'MR_MIME'

    def printNCharacters(N, paddingChar=' '):
        stringHere = ''
        for idx in range(N):
            stringHere+=paddingChar
        return stringHere

In [65]:
# FOSSILS AND GHOST NEED TO BE TREATED DIFFERENTLY
# I MAY SIMPLY SMASH THEM AS THE END AND CALL IT A DAY

# pokemon_constants

In [13]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/constants/pokemon_constants.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

names_Capital_list = []
names_Capital_fossiless_list = []

for oneline in linesFromFile:
    if ('\tconst ' in oneline) and not ('NO_MON' in oneline):
        #print(oneline.rsplit()[1])
        namehere = oneline.rsplit()[1]
        names_Capital_list.append(namehere)
        if not (namehere in listFossilsGhost_Capital):
            names_Capital_fossiless_list.append(namehere)

# pokedex_constants

In [14]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/constants/pokedex_constants.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

for oneline in linesFromFile:
    if ('\tconst DEX_' in oneline):
        if not oneline.rsplit()[1][4:] in names_Capital_list:
            print('ERROR!!! All mons in dex must have a constant (reverse is not true, fossils and ghost)')

# base_stats file

In [15]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/base_stats.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

names_allsmall_list = []

for oneline in linesFromFile:
    if ('INCLUDE "data/pokemon/base_stats/' in oneline):
        #print(oneline.rsplit()[1][len('"data/pokemon/base_stats/'):-len('.asm"')])
        names_allsmall_list.append(oneline.rsplit()[1][len('"data/pokemon/base_stats/'):-len('.asm"')])

# base_stats folder (big part)

In [16]:
inputRootPath = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/base_stats/'

HPs_list = []
ATK_list = []
DEF_list = []
SPD_list = []
SPC_list = []

Type1_list = []
Type2_list = []
CatchRate_list = []
BaseExp_list = []

names_PartlyCapital_list = []

Lvl1Moves_list = []
GrowthRate_list = []
TMHMMoves_list = []

for pkmn in names_allsmall_list:
    # open .asm file of that one pokemon, only case in which is 1 file per 1 pokemon
    inputFile = f'{inputRootPath}{pkmn}.asm'
    file_in = open(inputFile, 'r')
    linesFromFile = []
    for y in file_in.readlines():
        linesFromFile.append(y)

    # sanity check
    if not linesFromFile[0][len('\tdb DEX_'):-len(' ; pokedex id\n')]==names_Capital_fossiless_list[names_allsmall_list.index(pkmn)]:
        print('Dex idx does not match the pokemon name!')

    # grab base stats
    HPs_list.append(linesFromFile[2].rsplit()[1][:-1])
    ATK_list.append(linesFromFile[2].rsplit()[2][:-1])
    DEF_list.append(linesFromFile[2].rsplit()[3][:-1])
    SPD_list.append(linesFromFile[2].rsplit()[4][:-1])
    SPC_list.append(linesFromFile[2].rsplit()[5])

    # grab types, catch rate, base exp
    Type1_list.append(linesFromFile[5].rsplit()[1][:-1])
    Type2_list.append(linesFromFile[5].rsplit()[2])
    CatchRate_list.append(linesFromFile[6].rsplit()[1])
    BaseExp_list.append(linesFromFile[7].rsplit()[1])

    # grab Partly Capital name
    names_PartlyCapital_list.append(linesFromFile[10].rsplit()[1][:-len('PicFront,')])

    # grab level-1 learnset
    Lvl1Moves_list.append(linesFromFile[12][len('\tdb '):-len(' ; level 1 learnset\n')])

    # grab growth rate
    GrowthRate_list.append(linesFromFile[13][len('\tdb '):-len(' ; growth rate\n')])

    # grab tm/hm learnset
    TMHMMovesHere = ''
    TMHMTitleLineNumber = 15
    TMHMEndLineNumber = linesFromFile.index('\t; end\n')
    if not linesFromFile[TMHMTitleLineNumber+1]=='\ttmhm\n': # if it's not that no TM/HM learnable, e.g. Magikarp, Caterpie...
        lineNumberhere = TMHMTitleLineNumber+1
        while lineNumberhere<TMHMEndLineNumber:
            for maybeMove in linesFromFile[lineNumberhere].rsplit():
                if (maybeMove!='tmhm') and (maybeMove!='\\'): # the first is relevant only in the first line, and the second is not in the last
                    if ',' in maybeMove:
                        maybeMove += ' '
                    TMHMMovesHere += maybeMove
            lineNumberhere+=1
    TMHMMoves_list.append(TMHMMovesHere)
        

# evos_moves (another big part)

In [70]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/evos_moves.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

EvoInfo_list = []
LevelUpMoves_list = []

ignoreLvlUpMovesAlreadyDeleted = True

for Pkmn in names_PartlyCapital_list:
#    if (Pkmn=='Eevee'): # Mewtwo Ivysaur Eevee Machoke
#    if (Pkmn=='Mewtwo') or (Pkmn=='Ivysaur') or (Pkmn=='Eevee') or (Pkmn=='Machoke') or (Pkmn=='Ditto') or (Pkmn=='Caterpie'): # Mewtwo Ivysaur Eevee Machoke
    if True:
        infoPkmnhere = grabEvosMovesMon(Pkmn, linesFromFile)
        
        # grab evolution info
        EvolutionsProcessesHere = ''
        if (infoPkmnhere[1]=='; Evolutions\n') and (infoPkmnhere[2]=='\tdb 0\n'): # mon has no evolutions
            _ # do nothing?
        else:
            lineNumberHere = 2
            while infoPkmnhere[lineNumberHere]!='\tdb 0\n':
                infoEvolutionHere = infoPkmnhere[lineNumberHere][len('\tdb '):-len('\n')]
                EvolutionsProcessesHere += (infoEvolutionHere + ' - ')
                lineNumberHere+=1
        if len(EvolutionsProcessesHere)>0:
            EvolutionsProcessesHere = EvolutionsProcessesHere[:-3]
        EvoInfo_list.append(EvolutionsProcessesHere)
            
        # grab level-up learnset
        LevelUpLearnsetHere = ''
        firstLineMoves = infoPkmnhere.index('; Learnset\n')+1
        currentLine = firstLineMoves
        lastLineMoves = infoPkmnhere.index('\n')-2
        while currentLine<=lastLineMoves:
            infoPkmnhere[currentLine]
            if ignoreLvlUpMovesAlreadyDeleted and (infoPkmnhere[currentLine][:len(';\tdb ')]==';\tdb '):
                if currentLine==lastLineMoves:
                    LevelUpLearnsetHere = LevelUpLearnsetHere[:-3]
            else:
                LevelUpLearnsetHere += infoPkmnhere[currentLine].rsplit()[1][:-1]+' '+infoPkmnhere[currentLine].rsplit()[2]
                if currentLine<lastLineMoves:
                    LevelUpLearnsetHere += ' - '
            currentLine+=1
        LevelUpMoves_list.append(LevelUpLearnsetHere)
        

# cries

In [71]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/cries.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

cries_list = []

for oneline in linesFromFile:
    if ('mon_cry SFX_CRY_' in oneline) and not ('MissingNo' in oneline):
        cries_list.append(oneline.rsplit()[1]+' '+oneline.rsplit()[2]+' '+oneline.rsplit()[3])
        

# menu_icons

In [72]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/menu_icons.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

menuIcons_list = []

for oneline in linesFromFile:
    if ('nybble ICON_' in oneline) and not ('MissingNo' in oneline):
        menuIcons_list.append(oneline.rsplit()[1])
        

# palettes

In [73]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/palettes.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

palettes_list = []

for oneline in linesFromFile:
    if ('db PAL_' in oneline) and not ('MISSINGNO' in oneline):
        palettes_list.append(oneline.rsplit()[1])
        

# dex_entries

In [74]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_entries.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

species_list = []
height_dm_list = []
weight_hg_list = []

for Pkmn in names_PartlyCapital_list:
    species, height_dm, weight_hg = grabDexEntries(Pkmn, linesFromFile)
    species_list.append(species)
    height_dm_list.append(height_dm)
    weight_hg_list.append(weight_hg)

# dex_text

In [75]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_text.asm'

file_in = open(inputFile, 'r')
linesFromFile = []
for y in file_in.readlines():
#    if y!='\n':
#        print(y)
    linesFromFile.append(y)

dexTexts_list = []

for Pkmn in names_PartlyCapital_list:
    dexTexts_list.append(grabDexText(Pkmn, linesFromFile))

# Export to Excel-like

In [76]:
if checkAllSameLength([names_Capital_fossiless_list, names_allsmall_list, HPs_list, ATK_list, DEF_list, SPD_list, SPC_list, Type1_list, Type2_list, CatchRate_list, BaseExp_list, names_PartlyCapital_list, Lvl1Moves_list, GrowthRate_list, TMHMMoves_list, EvoInfo_list, LevelUpMoves_list, cries_list, menuIcons_list, palettes_list, species_list, height_dm_list, weight_hg_list, dexTexts_list]):
    pokemonData_temp = {
                        'Name_Capital':names_Capital_fossiless_list,
                        'Name_allsmall':names_allsmall_list,
                        'Name_PartlyCapital':names_PartlyCapital_list,
                        'HPs':HPs_list,
                        'ATK':ATK_list,
                        'DEF':DEF_list,
                        'SPC':SPC_list,
                        'SPD':SPD_list,
                        'Type1':Type1_list,
                        'Type2':Type2_list,
                        'CatchRate':CatchRate_list,
                        'BaseExp':BaseExp_list,
                        'Lvl1Moves':Lvl1Moves_list,
                        'GrowthRate':GrowthRate_list,
                        'TMHMMoves':TMHMMoves_list,
                        'EvoInfo':EvoInfo_list,
                        'LevelUpMoves':LevelUpMoves_list,
                        'cries':cries_list,
                        'menuIcons':menuIcons_list,
                        'palettes':palettes_list,
                        'species':species_list,
                        'height_dm':height_dm_list,
                        'weight_hg':weight_hg_list,
                        'dexTexts':dexTexts_list
                       }
    dfPokemonData = pd.DataFrame(data=pokemonData_temp)

    with pd.ExcelWriter('/Users/Michael/Desktop/Core/Programmi_miei/Pokemon/pretStuff/outputPokemonInfoToExcel/testPokemonToExcel.ods') as writer:
        dfPokemonData.to_excel(writer)

    dfPokemonData.head()